In [ ]:
import pandas as p
url = 'https://etherscan.io/txsInternal?a=0xea3f1b409f5e9211eed087c9f5f621d1249b3e7b'
from qore import *
xp = XPath()
xcols = {
    #'to'      : '/html/body/div[1]/div[4]/div[2]/div/div/div/table/tbody/tr/td[6]/span/a/text()',
    'value'   : '/html/body/div[1]/div[4]/div[2]/div/div/div/table/tbody/tr/td[7]/text()',
}
res = xp.xpath2df(url, xcols)
p.DataFrame(res)

In [ ]:
%pylab inline
import numpy as n
from bitmex import apiRequest

%reload_ext autoreload
%autoreload 2
%pylab inline
from qoreliquid import *
import numpy as n
from bitmex import CoinMarketCap
from pylab import rcParams
rcParams['figure.figsize'] = 20, 5
import seaborn as sns
sns.set()
cmc = CoinMarketCap()

In [ ]:
dft = cmc.getAllTokens(tokens=False)
dft
#cmc.tokens

In [ ]:
dft = cmc.getAllTokens(tokens=True)
dft
#cmc.tokens

In [ ]:
from bitmex import PortfolioModeler
pm = PortfolioModeler()
df = pm.generatePortfolioT1Supply(df)
df#['id']

In [ ]:
#df = df.head(300)
#df[fi] = map(lambda x: x.decode('utf-8').strip(), df[fi])
#pp = sns.pairplot(df)#, hue=fi)
#plt.show()

In [ ]:
# show portfolio eth1 charts
port = 'XRP STEEM PRO BNB PPP BRAT BCD SALT KIN PLR RHOC PPT CVC CDT NEWB BAS PAY EOS DOM XRL LINK TNT BAT DNT SNT SKIN FUCK NIO HMQ IXT MSP IND DENT FUN BET'
li = list(df.loc[port.split(' '),['id']]['id'])
#print ' '.join(li)
for i in list(li):
    try:
        cmc.glo(i)
        plt.show()
    except: ''

In [ ]:
# show portfolio eth2 charts
port = 'CHC BCC XRL RHOC KIN PPT SALT CDT BRAT PAY MBRS XRL EOS TAAS BNB HMQ OMG WINGS POE CTR LINK SKIN FUCK BAT CVC MCO SNT PPP CAT DOM BBT BET STX TIX DNT MGO VERI ICN NIO BQX TNT ZRX IXT MSP IND FUN LRC PLR'
li = list(df.loc[port.split(' '),['id']]['id'])
#print ' '.join(li)
for i in list(li):
    try:
        cmc.glo(i)
        plt.show()
    except: ''

In [ ]:
nu = 30
for i in list(df.head(nu)['id']):
    cmc.glo(i)
    plt.show()
print '=-=-=-=-=-=-=-=-'
print '=-=-=-=-=-=-=-=-'
for i in list(df.tail(nu)['id']):
    cmc.glo(i)
    plt.show()

In [ ]:
#for i in 'bitcoin ethereum kin'.split(' '):
# b3 == pump n dump example
for i in 'b3coin'.split(' '):
    cmc.glo(i)